###### TODO Recording



- Go to the event hub namespace that we have already created
- loony-eventhub-namespace

Create eventhub

- Click on +Event hub
- Add following setails

	name : loony-movies

- Click on create
- Event hub has created in the namespace
- We can see the eventhub at the bottom in the namespace page

- We need two details from this page now
	name : loony-movies
	primary key

- To get the primary key go to 'Shared access policies' which is there in the left side of the namespace page
- Then click on the 'RootManageSharedAccessKey' 
- Then the right side keys will be active
- Copy the connection string primary key

	Connection string–primary key:
	Endpoint=sb://loony-eventhub-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=eKnj1tqnC6UJdhA9I2IVN1rvh2oVcc6Ubd2Cnvr4PQ0=

- Go to the Databricks workspace
- Have StreamingStreamingJoins open on one tab
- Have MoviesRatingsSource open on another tab
- Start with the code in StreamingStreamingJoins

In [0]:
import json

from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, from_json

In [0]:
primaryKey_movies = "Endpoint=sb://loony-eventhub-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=eKnj1tqnC6UJdhA9I2IVN1rvh2oVcc6Ubd2Cnvr4PQ0="
entityPath_movies = "EntityPath=loony-movies"

connectionString_movies = primaryKey_movies + ";" + entityPath_movies

ehConf_movies = {}

startingEventPosition_movies = {
  "offset": "-1",  
  "seqNo": -1,            
  "enqueuedTime": None,  
  "isInclusive": True
}

ehConf_movies['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString_movies)
ehConf_movies["eventhubs.startingPosition"] = json.dumps(startingEventPosition_movies)

In [0]:
primaryKey_ratings = "Endpoint=sb://loony-eventhub-namespace.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=eKnj1tqnC6UJdhA9I2IVN1rvh2oVcc6Ubd2Cnvr4PQ0="
entityPath_ratings = "EntityPath=loony-ratings"

connectionString_ratings = primaryKey_ratings + ";" + entityPath_ratings

ehConf_ratings = {}

startingEventPosition_ratings = {
  "offset": "-1",  
  "seqNo": -1,            
  "enqueuedTime": None,  
  "isInclusive": True
}

ehConf_ratings['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString_ratings)
ehConf_ratings["eventhubs.startingPosition"] = json.dumps(startingEventPosition_ratings)

In [0]:
streaming_schema_movies = StructType([StructField("name", StringType(), True),
                                      StructField("year", IntegerType(), True),
                                      StructField("director", StringType(), True),
                                      StructField("writer", StringType(), True),
                                      StructField("star", StringType(), True)
                                     ])

In [0]:
streaming_data_movies = spark.readStream \
                             .format("eventhubs") \
                             .options(**ehConf_movies) \
                             .load()

In [0]:
streaming_data_movies = streaming_data_movies.select(from_json(col("body").cast("string"), streaming_schema_movies)) \
                                             .withColumnRenamed("from_json(CAST(body AS STRING))", "data") \
                                             .select(col('data.*'))   

streaming_data_movies.display()

name,year,director,writer,star
The Shining,1980,Stanley Kubrick,Stephen King,Jack Nicholson
The Blue Lagoon,1980,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill
Airplane!,1980,Jim Abrahams,Jim Abrahams,Robert Hays
Caddyshack,1980,Harold Ramis,Brian Doyle-Murray,Chevy Chase
The Long Riders,1980,Walter Hill,Bill Bryden,David Carradine
Any Which Way You Can,1980,Buddy Van Horn,Stanford Sherman,Clint Eastwood
The Gods Must Be Crazy,1980,Jamie Uys,Jamie Uys,N!xau
Popeye,1980,Robert Altman,Jules Feiffer,Robin Williams
Ordinary People,1980,Robert Redford,Judith Guest,Donald Sutherland


##### TODO Recording

- Let's show some data in this stream before we move to the ratings stream (the ratings stream will already have some data)
- Switch over to the MoviesRatingsSource
- Run till the command which has run_movies()

The movies added are:

        event_data_batch.add(EventData('{"name":"The Shining","year":1980,"director":"Stanley Kubrick","writer":"Stephen King","star":"Jack Nicholson"}'))
        event_data_batch.add(EventData('{"name":"The Blue Lagoon","year":1980,"director":"Randal Kleiser","writer":"Henry De Vere Stacpoole","star":"Brooke Shields"}'))
        event_data_batch.add(EventData('{"name":"Star Wars: Episode V - The Empire Strikes Back","year":1980,"director":"Irvin Kershner","writer":"Leigh Brackett","star":"Mark Hamill"}'))
        event_data_batch.add(EventData('{"name":"Airplane!","year":1980,"director":"Jim Abrahams","writer":"Jim Abrahams","star":"Robert Hays"}'))
        event_data_batch.add(EventData('{"name":"Caddyshack","year":1980,"director":"Harold Ramis","writer":"Brian Doyle-Murray","star":"Chevy Chase"}'))

####### TODO Recording

- Run the cells below
- Please note that there will already be records in this stream because of the previous demo

In [0]:
streaming_schema_ratings = StructType([StructField("name", StringType(), True),
                                       StructField("rating", StringType(), True),
                                       StructField("score", FloatType(), True)
                                      ])

In [0]:
streaming_data_ratings = spark.readStream \
                             .format("eventhubs") \
                             .options(**ehConf_ratings) \
                             .load()

In [0]:
streaming_data_ratings = streaming_data_ratings.select(from_json(col("body").cast("string"), streaming_schema_ratings)) \
                                               .withColumnRenamed("from_json(CAST(body AS STRING))", "data") \
                                               .select(col('data.*'))   

streaming_data_ratings.display()

name,rating,score
The Shining,R,8.4
The Blue Lagoon,R,5.8
Star Wars: Episode V - The Empire Strikes Back,PG,8.7
Airplane!,PG,7.7
Caddyshack,R,7.3
Friday the 13th,R,6.4
The Blues Brothers,R,7.9
Raging Bull,R,8.2
Lagaan,PG,9.2
Star Wars: Episode V - The Empire Strikes Back,PG,8.2


In [0]:
inner_join = streaming_data_movies.join(streaming_data_ratings, on=["name"])

inner_join.display()

name,year,director,writer,star,rating,score
The Shining,1980,Stanley Kubrick,Stephen King,Jack Nicholson,R,8.4
Airplane!,1980,Jim Abrahams,Jim Abrahams,Robert Hays,PG,7.7
Caddyshack,1980,Harold Ramis,Brian Doyle-Murray,Chevy Chase,R,7.3
The Blue Lagoon,1980,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,R,5.8
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill,PG,8.7
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill,PG,8.2
Star Wars: Episode V - The Empire Strikes Back,1980,Irvin Kershner,Leigh Brackett,Mark Hamill,PG,8.2
Ordinary People,1980,Robert Redford,Judith Guest,Donald Sutherland,PG,8.1
The Long Riders,1980,Walter Hill,Bill Bryden,David Carradine,PG,7.1
The Gods Must Be Crazy,1980,Jamie Uys,Jamie Uys,N!xau,PG,8.6


###### TODO Recording

- Add some more data to the movies eventhub and wait for a bit

        event_data_batch.add(EventData('{"name":"The Long Riders","year":1980,"director":"Walter Hill","writer":"Bill Bryden","star":"David Carradine"}'))
        event_data_batch.add(EventData('{"name":"Any Which Way You Can","year":1980,"director":"Buddy Van Horn","writer":"Stanford Sherman","star":"Clint Eastwood"}'))
        event_data_batch.add(EventData('{"name":"The Gods Must Be Crazy","year":1980,"director":"Jamie Uys","writer":"Jamie Uys","star":"N!xau"}'))
        event_data_batch.add(EventData('{"name":"Popeye","year":1980,"director":"Robert Altman","writer":"Jules Feiffer","star":"Robin Williams"}'))
        event_data_batch.add(EventData('{"name":"Ordinary People","year":1980,"director":"Robert Redford","writer":"Judith Guest","star":"Donald Sutherland"}'))
        
- Scroll to the top where we display streaming_data_movies and show that new rows have been read in
- Scroll down and show that the result of the join is not updated since none of the new movies added have ratings

###### TODO Recording

- Add some data to the loony-ratings event hub

        event_data_batch.add(EventData('{ "name": "Amar Akbar Anthony", "rating": "PG", "score": 9.3}'))
        event_data_batch.add(EventData('{ "name": "The Long Riders","rating": "PG", "score": 7.1 }'))
        event_data_batch.add(EventData('{ "name": "The Gods Must Be Crazy", "rating": "PG", "score": 8.6}'))
        event_data_batch.add(EventData('{ "name": "Ordinary People", "rating": "PG", "score": 8.1}'))

- Scroll up and show that the streaming_data_ratings stream now has new records
- Scroll down and show that there are new entries in the stream-stream join

In [0]:
join_and_filter = streaming_data_ratings.join(streaming_data_movies, on=["name"]) \
                                .where(streaming_data_ratings.rating == "PG")

join_and_filter.display()

name,rating,score,year,director,writer,star
Ordinary People,PG,8.1,1980,Robert Redford,Judith Guest,Donald Sutherland
The Long Riders,PG,7.1,1980,Walter Hill,Bill Bryden,David Carradine
The Gods Must Be Crazy,PG,8.6,1980,Jamie Uys,Jamie Uys,N!xau
Airplane!,PG,7.7,1980,Jim Abrahams,Jim Abrahams,Robert Hays
Star Wars: Episode V - The Empire Strikes Back,PG,8.7,1980,Irvin Kershner,Leigh Brackett,Mark Hamill
Star Wars: Episode V - The Empire Strikes Back,PG,8.2,1980,Irvin Kershner,Leigh Brackett,Mark Hamill
Star Wars: Episode V - The Empire Strikes Back,PG,8.2,1980,Irvin Kershner,Leigh Brackett,Mark Hamill


In [0]:
not_supported_join = streaming_data_movies.join(streaming_data_ratings, on=["name"], how="leftOuter")

not_supported_join.display()

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3541484190323637> in <module> 
 1 not_supported_join = streaming_data_movies . join ( streaming_data_ratings , on = [ "name" ] , how = "leftOuter" ) 
 2 
 ----> 3 not_supported_join . display ( ) 

 /databricks/python_shell/dbruntime/monkey_patches.py in df_display (df, *args, **kwargs) 
 27 df . display ( ) is an alias for display ( df ) . Run help ( display ) for more information . 
 28 """
 ---> 29 display ( df , * args , ** kwargs ) 
 30 
 31 @ when_imported ( 'pyspark.sql' ) 

 /databricks/python_shell/scripts/PythonShellImpl.py in display (self, input, *args, **kwargs) 
 1120 # exists and then if it does, whether it is actually streaming. 
 1121 if hasattr ( input , 'isStreaming' ) and input . isStreaming : 
 -> 1122 handleStreamingDataFrame ( input , self . sc , self . sqlContext , self . entry_point , kwargs ) 
 1123 else : 
 1124 if kwargs . get ( 'streamName' ) : 

 /databricks/python_shell/dbruntime/display.py in handleStreamingDataFrame (input, sc, sqlContext, entry_point, kwargs) 
 109 .DisplayHelper.getStreamName())
 110 
 --> 111 entry_point.getDriverSparkHooks().displayStreamingDataFrame(input._jdf, name, trigger,
 112 kwargs.get('checkpointLocation'))
 113 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Stream-stream LeftOuter join between two streaming DataFrame/Datasets is not supported without a watermark in the join keys, or a watermark on the nullable side and an appropriate range condition;
Join LeftOuter, (name#92373 = name#101479)
:- Project [data#92371.name AS name#92373, data#92371.year AS year#92374, data#92371.director AS director#92375, data#92371.writer AS writer#92376, data#92371.star AS star#92377]
: +- Project [from_json(CAST(body AS STRING))#92369 AS data#92371]
: +- Project [from_json(StructField(name,StringType,true), StructField(year,IntegerType,true), StructField(director,StringType,true), StructField(writer,StringType,true), StructField(star,StringType,true), cast(body#90388 as string), Some(Etc/UTC)) AS from_json(CAST(body AS STRING))#92369]
: +- Project [cast(body#89953 as string) AS body#90388]
: +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@650f5b94,eventhubs,List(),None,List(),None,Map(eventhubs.connectionString -> P4w5ooxK264fEn4fSjKMh0CgCpv6wQLbtLTOzbqmvtyM/MK1uyEyPEhrqFUohdaoEq1ipYfu2zWSoFKQ6knyBLS/5gUDr3ADZyLcTE6KvSMk7p/pKjtZca2Dq+6rahexk0vGX03dVL6MMXNSFPbwoq890m5/e3/SJ67TeuugVvq2PEnOvg2JIssGDiJmDM+V2WTcgrC+wMZv9qyZjhhsR5bJ26rx9e/YBFu5uAaZhHIPbE6JO3DTP19K6aNRDV4y8ClY9u8KFRal/Ix/sXaqvA==, eventhubs.startingPosition -> {"offset": "-1", "seqNo": -1, "enqueuedTime": null, "isInclusive": true}),None), eventhubs, [body#89953, partition#89954, offset#89955, sequenceNumber#89956L, enqueuedTime#89957, publisher#89958, partitionKey#89959, properties#89960, systemProperties#89961]
+- Project [data#101477.name AS name#101479, data#101477.rating AS rating#101480, data#101477.score AS score#101481]
 +- Project [from_json(CAST(body AS STRING))#101475 AS data#101477]
 +- Project [from_json(StructField(name,StringType,true), StructField(rating,StringType,true), StructField(score,FloatType,true), cast(body#101373 as string), Some(Etc/UTC)) AS from_json(CAST(body AS STRING))#101475]
 +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@650f5b94,eventhubs,List(),None,List(),None,Map(eventhubs.connectionString -> P4w5ooxK264fEn4fSjKMh0CgCpv6wQLbtLTOzbqm

In [0]:
# Next demo we will show joins with watermark